In [3]:
import pickle

In [4]:
training_data_7k = []
with open('training_data_7k.pkl' ,'rb')as f :
    training_data_7k = pickle.load(f)

In [5]:
# training_data_7k[0]

In [6]:
import multiprocessing
from gensim.models import Word2Vec
import time
import numpy as np
from numpy.linalg import norm

In [7]:
cores = multiprocessing.cpu_count()
print('Number of cores in this device {}'.format(cores))

Number of cores in this device 8


In [8]:
docs_num = len(training_data_7k)
tokens_num = sum([len(x) for x in training_data_7k])
print('Number of all docs {}'.format(docs_num))
print('Number of all Tokens {}'.format(tokens_num))

Number of all docs 7562
Number of all Tokens 1467075


In [9]:
%%time
w2v_model = Word2Vec(min_count=1 , window=15 , vector_size=300 , alpha=0.03 , workers=cores-1)
w2v_model.build_vocab(training_data_7k)
w2v_model_vocab_size = len(w2v_model.wv)
print('Vocab size : ' , w2v_model_vocab_size)

Vocab size :  40507
Wall time: 2.49 s


In [10]:
# start = time.time()
# w2v_model.train(training_data_7k , total_examples=w2v_model.corpus_count , epochs= 30)
# end = time.time()
# print("{} s". format(end-start))

In [11]:
docs_tfidf_11k = []
with open('tfidf_50k.pkl' ,'rb')as f :
    docs_tfidf_11k = pickle.load(f)

In [12]:
w2v_model = Word2Vec.load("w2v_7k_300d.model")

In [13]:
docs_tfidf_7k = []
with open('tfidf_7k.pkl' ,'rb')as f :
    docs_tfidf_7k = pickle.load(f)

In [14]:
# docs_tfidf_7k[0]

In [15]:
def make_docs_embeding(docs_tfidf):
    docs_embedding = []
    for doc in docs_tfidf:
        doc_vector = np.zeros(300)
        weight_sum = 0
        if(len(doc.items())!=0):
            
       
            for token , weight in doc.items() :


                        doc_vector += w2v_model.wv[token] * weight
                        weight_sum += weight

            docs_embedding.append(doc_vector/weight_sum)
        
        
    
 
        
    return docs_embedding

In [16]:
docs_embedding_7k = make_docs_embeding(docs_tfidf_7k)
with open('docs_embedding_7k.pkl' , 'wb') as f :
        pickle.dump(docs_embedding_7k,f)

In [17]:
def similarity(doc1 , doc2) :
    similarity_score = np.dot(doc1 , doc2) / ((norm(doc1)) *(norm(doc2)) )
    return (similarity_score +1)/2

In [18]:
s = similarity(docs_embedding_7k[11] , docs_embedding_7k[10])
s

0.84620501630821

# KNN implementation

In [25]:
import xlrd
data_set = '50k_combined.xlsm'
data_reader = xlrd.open_workbook(data_set)
content = data_reader.sheet_by_index(0)

In [23]:
def KNN (k , train_embedding , test_embedding) :
    #sport , economic , political , culture , health
    classes = [[],[],[],[],[]]
    
    for index ,test_doc in enumerate(test_embedding) :
        distances = {}
        for i , train_doc in enumerate(train_embedding) :
            dist = np.linalg.norm(test_doc-train_doc)
#               sim = similarity(test_doc , train_doc)
            
            distances[i] = dist
        
        distances = dict(sorted(distances.items(), key=lambda item: item[1]))
        
        
        k_first = dict(list(distances.items())[0: k])
        
        categories_repeat = {}
        
        for doc in k_first :
            cat = data_reader.sheet_by_index(0).cell(int(doc), 2).value
            if(cat not in categories_repeat) :
                categories_repeat[cat] = 1
            else :
                categories_repeat[cat] += 1
        
        categories_repeat = dict(sorted(categories_repeat.items(), key=lambda item: item[1], reverse=True))
        best_category= list(categories_repeat)[0]
        
#         print(best_category)
        
        if(best_category == 'sport') :
            classes[0].append((test_doc , index))
        elif(best_category == 'economy') :
            classes[1].append((test_doc , index))
        elif(best_category == 'politics') :
            classes[2].append((test_doc , index))
        elif(best_category == 'culture') :
            classes[3].append((test_doc , index))
        elif(best_category == 'health') :  
            classes[4].append((test_doc , index))


                
    classificitaion_7k={'classes' : classes}
    with open('classification_7k_3.pkl' , 'wb') as f :
        pickle.dump(classificitaion_7k,f)
    
        

In [21]:
with open('docs_embedding_50k.pkl' ,'rb')as f :
    docs_embedding_50k = pickle.load(f)

In [24]:
%%time
KNN (3 , docs_embedding_50k , docs_embedding_7k)

Wall time: 1h 2min 59s


## comparison between k

In [37]:
data_set2 = 'IR1_7k_news.xlsx'
data_reader2 = xlrd.open_workbook(data_set2)
content = data_reader2.sheet_by_index(0)

with open('classification_7k_3.pkl' ,'rb')as f :
    classification_7k_3 = pickle.load(f)

classes = classification_7k_3['classes']
for doc , index in classes[0] :
        
        url =  data_reader2.sheet_by_index(0).cell(int(index), 1).value
       
        print(url)
        print("================================================")
    



https://www.farsnews.ir/news/14000803000698/هفته-دوم-لیگ-برتر|-گل-گهر-و-پیکان-پیروز-شدند-دومین-باخت‌-فجرسپاسی-و
https://www.farsnews.ir/news/14000803000751/بازیکنان-پدیده-باز-هم-تمرین-نکردند-مهاجری-استعفا-کرد-میثاقیان-در
https://www.farsnews.ir/news/14000803000358/مقدماتی-قهرمانی-زیر-23-سال-آسیا|-اعلام-ترکیب-تیم-امید-مقابل-نپال
https://www.farsnews.ir/news/14000803000108/جیمی-جامپ-چگونه-مانع-از-حمله-مسی-در-ال-کلاسیکوی-فرانسه-شد-فیلم
https://www.farsnews.ir/news/14000803000093/بررسی-عملکرد-عابدزاده-|-دروازه‌بان-کشورمان-ضعیف‌ترین-بازیکن-شد-فیلم
https://www.farsnews.ir/news/14000802000306/گل-رونالدینیویی-ستاره-رقیب-ایران-در-لیگ-امارات-فیلم
https://www.farsnews.ir/news/14000802000141/ال-کلاسیکو-با-رقابت-جانشینان-مسی-و-رونالدو-عکس
https://www.farsnews.ir/news/14000801000761/ربیعی-با-تیمی-بازی-داریم-که-از-لحاظ-انگیزشی-بالاتر-از-حالت-عادی-است
https://www.farsnews.ir/news/14000801000728/رکوردزنی-بایرنی-ها-ادامه-دارد-نویر-در-آستانه-شکستن-رکورد-اولیورکان
https://www.farsnews.ir/news/140008010006

In [39]:
with open('classification_7k_5.pkl' ,'rb')as f :
    classification_7k_5 = pickle.load(f)

classes = classification_7k_5['classes']
for doc , index in classes[0] :
        url =  data_reader2.sheet_by_index(0).cell(int(index), 1).value
       
        print(url)
        print("================================================")

https://www.farsnews.ir/news/14000803000860/رودنیل-مدافع-استقلال-مرتکب-خطای-پنالتی-نشد-داور-قضاوت-خوبی-انجام-داد
https://www.farsnews.ir/news/14000803000751/بازیکنان-پدیده-باز-هم-تمرین-نکردند-مهاجری-استعفا-کرد-میثاقیان-در
https://www.farsnews.ir/news/14000803000582/وینگر-تیم-ملی-جزو-بهترین‌های-لیگ-بلژیک-عکس
https://www.farsnews.ir/news/14000801000728/رکوردزنی-بایرنی-ها-ادامه-دارد-نویر-در-آستانه-شکستن-رکورد-اولیورکان
https://www.farsnews.ir/news/14000801000611/پاسخ-سازمان-لیگ-به-بیانیه-باشگاه-استقلال
https://www.farsnews.ir/news/14000730000394/مهرام-پدافند-تهران-را-از-کار-انداخت
https://www.farsnews.ir/news/14000729000548/لیگ-برتر-فوتسال-صدرنشینی-مس-با-شکست-فرش-آرا-پیروزی-پرگل-سپاهان-مقابل
https://www.farsnews.ir/news/14000729000414/هافبک-استقلال-شاگرد-عنایتی-شد-عکس
https://www.farsnews.ir/news/14000729000336/براتی-افکار-عمومی-نظر-کاملا-مثبتی-در-خصوص-عملکرد-فدراسیون-فوتبال-دارد
https://www.farsnews.ir/news/14000729000132/اعلام-اسامی-داوران-هفته-دوم-لیگ-برتر-فوتبال
https://www.farsnews.i

In [41]:
with open('classification_7k_7.pkl' ,'rb')as f :
    classification_7k_7 = pickle.load(f)

classes = classification_7k_7['classes']
for doc , index in classes[4] :
        url =  data_reader2.sheet_by_index(0).cell(int(index), 1).value
       
        print(url)
        print("================================================")

url
https://www.farsnews.ir/news/14000803000905/کاپیتان-ذوب‌آهن-‌-حق‌مان-باخت-نبود-شاید-گل-دوم-استقلال-آفساید-بود
https://www.farsnews.ir/news/14000803000852/کریمی-ما-بیشتر-از-استقلال-و-پرسپولیس-به-خاطر-نبود-هوادار-ضرر-کردیم
https://www.farsnews.ir/news/14000803000887/مهدی-پور-برای-استقلال-3-امتیاز-بازی-مهم-بود-یامگا-بازیکن-تاثیرگذار-و
https://www.farsnews.ir/news/14000803000778/کوثری-کسانی-که-به-دنبال-فوتبال-هستند-از-جادوگری-پرهیز-می‌کنند-فعلا
https://www.farsnews.ir/news/14000803000655/حاشیه-بازی-فجرسپاسی-و-گل‌گهر|-دو-بازیکن-تیم-شیرازی-از-ترکیب-خارج-شدند-
https://www.farsnews.ir/news/14000802000428/تحقیر-منچستر-به-دست-لیورپول-یک-نفر-به-داد-شیاطین-سرخ-برسد
https://www.farsnews.ir/news/14000802000394/پیروزی-درخشان-آاک-در-شب-گلزنی-لژیونرهای-ایرانی-تیم-آتنی-موقتا-صدرنشین
https://www.farsnews.ir/news/14000802000340/اطلاعیه-باشگاه-پرسپولیس-در-خصوص-تبلیغات-محیطی
https://www.farsnews.ir/news/14000802000251/عجیب-ولی-واقعی-فروش-پیراهن-مسی-قبل-از-ال-کلاسیکو-عکس
https://www.farsnews.ir/news/1400

In [43]:
import hazm
hazm_normalaizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()

In [44]:
from collections import Counter
import math
def query_embeding(query ,docs_num) :
       
        normal_query = hazm_normalaizer.normalize(query)
        tokens = hazm.word_tokenize(normal_query)
        tokens_stems = []
        for t in tokens :
            stem_token = lemmatizer.lemmatize(t)
            if("#" in stem_token):
                mazi , mozare = stem_token.split("#")
                tokens_stems.append(mazi)
                tokens_stems.append(mozare)
            else :
                tokens_stems.append(stem_token)
            
        token_tfidf = {}

        
        for term, count in Counter(tokens_stems).items():
            tf = (1 + math.log10(count))
            idf = math.log10(docs_num/1)
            tfidf = tf * idf
            token_tfidf [term] = tfidf
        
        query_vector = np.zeros(300)
        weight_sum = 0
        for token , weight in token_tfidf.items() :
            try:
            
                query_vector += w2v_model.wv[token] * weight
                
            except KeyError:
                continue
            weight_sum += weight
        if weight_sum == 0 :
            return query_vector
 
     
        return query_vector/weight_sum

In [45]:
def answer_query(query , cat , k_top):
    data_set2 = 'IR1_7k_news.xlsx'
    data_reader2 = xlrd.open_workbook(data_set2)
    content = data_reader2.sheet_by_index(0)
    with open('classification_7k_7.pkl' ,'rb')as f :
        classification_7k = pickle.load(f)
    f.close()
    
    classes = classification_7k['classes']
    
        
    if(cat == 'sport') :
        index_search = 0
    elif(cat == 'economy') :
        index_search = 1
    elif(cat == 'politics') :
        index_search = 2
    
    elif(cat == 'culture') : 
        index_search = 3
    elif(cat == 'health') : 
        index_search = 4
    
    query_embed = query_embeding(query , 7562)
    answers = {}
    
    for doc , index in classes[index_search] :
        sim = similarity(doc , query_embed) 
        answers[index] = sim
    
    answers = dict(sorted(answers.items(), key=lambda item: item[1], reverse=True))
    k_first = dict(list(answers.items())[0:k_top])
    
    for d in k_first:
        print("{} : {}".format(d,k_first[d]))
        title = data_reader2.sheet_by_index(0).cell(int(d), 2).value
        url =  data_reader2.sheet_by_index(0).cell(int(d), 1).value
        print(title)
        print(url)
        print("================================================")
            
        
    
    

In [54]:
query_with_cat = input('Enter your query first then with space enter yor category like cat:your category')
split_query = query_with_cat.split('cat')
query = split_query[0]
category = split_query[1].replace(":" ,"")

answer_query(query , category , 20)


Enter your query first then with space enter yor category like cat:your categoryدرگذشت پیشکسوت cat:culture
6379 : 0.8101335984244663
سعدی در ژاپن
https://www.isna.ir/news/1400020503308/سعدی-در-ژاپن
7308 : 0.7919871703699556
نارسایی‌های خط‌های نوشتاری
https://www.isna.ir/news/1400061913964/نارسایی-های-خط-های-نوشتاری
7506 : 0.7874725037958134
آنتن های رادیویی در جستجوی زندگی در سیاره ای دیگر
https://www.isna.ir/news/1400072115111/آنتن-های-رادیویی-در-جستجوی-زندگی-در-سیاره-ای-دیگر
2512 : 0.7864411214983731
حسینی:  امنیت خط قرمز ما است و نخواهیم گذاشت مرزهای ما به مخاطره بیفتد
https://www.farsnews.ir/news/14000710000693/حسینی--امنیت-خط-قرمز-ما-است-و-نخواهیم-گذاشت-مرزهای-ما-به-مخاطره-بیفتد
7518 : 0.7848805108015791
انتصاب سرپرست بنیاد شهید و امور ایثارگران استان خراسان‌شمالی
https://www.isna.ir/news/1400072417067/انتصاب-سرپرست-بنیاد-شهید-و-امور-ایثارگران-استان-خراسان-شمالی
3215 : 0.779886709082465
در سفر رئیس جمهور به خراسان جنوبی چه گذشت؟/ از درددل فعالان اقتصادی با آیت‌الله رئیسی تا مهلت  